<a href="https://colab.research.google.com/github/jehee-lee/jjtech/blob/main/mnist_CAM_RPi_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모델 생성, 매개변수 불러오기

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)
  
device = torch.device('cpu')
model = Net()
PATH = './model/'
model.load_state_dict(torch.load(PATH + 'model_for_RO_CNN.pt', map_location=device))
model.eval()

하드웨어 핀 설정 (GPIO) 및 비디오 설정

In [ ]:
import RPi.GPIO as GPIO
import cv2
from time import sleep as sl

led = 21
button = 16
GPIO.setmode(GPIO.BCM)

GPIO.setup(led,GPIO.OUT)
GPIO.setup(button,GPIO.IN,pull_up_down=GPIO.PUD_UP)
cap = cv2.VideoCapture(0)

frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
         int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

콜백함수 지정 및 인터럽트 실행문 선언

ANN과 달리 모델에 들어가는 input의 형태가 4 텐서이기 때문에

이미지를 [1, 1, 28, 28]텐서형으로 변경 ( => [5,1,10,10] conv신경망으로 읽힘 )

In [ ]:
print(frame_size)

def call_back_for_capture(channel):
    print("button pressed")
    GPIO.output(led,GPIO.HIGH)
    cv2.imwrite('./data/mnist.jpg',frame1)
    sl(1)
    img = cv2.imread('./data/mnist.jpg', cv2.IMREAD_GRAYSCALE)
    GPIO.output(led,GPIO.LOW)
    _, img1 = cv2.threshold(img,160, 255, cv2.THRESH_BINARY_INV)
    img2 = cv2.resize(img1, (28,28), interpolation = cv2.INTER_AREA)
    img3=torch.from_numpy(img2)
    result = img3.view(1,1,28,28).float().to(device) #코드 수정
    single_prediction = model(result)
    print('Prediction:', torch.argmax(single_prediction,1).item())
    plt.imshow(img3.view(28,28),cmap='Greys')
    plt.show()

GPIO.add_event_detect(button, GPIO.RISING,
                      callback=call_back_for_capture)

스트림 영상 출력
(esc를 누르면 종료)

In [ ]:

while(cap.isOpened()):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    frame = frame[500:1500,800:1800]
    frame1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    if (ret):
        frame2 = cv2.resize(frame1, dsize=(640, 480), interpolation=cv2.INTER_AREA)
        cv2.imshow('stream',frame2)
        cv2.moveWindow('stream',200,100)
        k = cv2.waitKey(1) & 0xFF
        if(k== 27):
            break;
cap.release()
cv2.destroyAllWindows()
GPIO.cleanup